# 네이버뉴스 본문, 댓글, 감정

**requests를 이용한 네이버 뉴스 정보 크롤링**

In [2]:
import re, csv, datetime, time, lxml, requests, os
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd 
from pandas import DataFrame, Series
import json


columns_news = ['검색어', '게재일', '게재시간', '매체명', '헤드라인', '본문', 'URL', '댓글 수', '좋아요 수', '슬퍼요 수', 
                '화나요 수', '훈훈해요 수', '후속기사 희망 수']
columns_comments = ['URL', '댓글작성자', '댓글', '공감 수', '비공감 수', '댓글작성시간']


df_news = pd.DataFrame(columns=columns_news)
df_comments = pd.DataFrame(columns=columns_comments)

header = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36',
    }

# 수집날짜 
s_date = "2019.09.19"
e_date = "2019.10.18"
s_from = s_date.replace(".", "")
e_to = e_date.replace(".", "")
page = 1

#수집 키워드
query = '이춘재 8차'

#기사 수집
def get_news(n_url):    
    news_detail = []
    res = requests.get(n_url)
    soup = BeautifulSoup(res.content, 'lxml')
    
    #게재일 및 게재시간 
    date_day = soup.select('.t11')[0].get_text()[:10]
    news_detail.append(date_day)
    date_time = soup.select('.t11')[0].get_text()[11:]
    news_detail.append(date_time)
    
    #매체명 
    media_outlet = soup.select('div.article_header > div.press_logo')[0].a.img.get('title')
    news_detail.append(media_outlet)

    #헤드라인 
    title = soup.select('h3#articleTitle')[0].text
    news_detail.append(title)

    #본문
    _text = soup.select('#articleBodyContents')[0].get_text().replace('\n', " ")
    body_text = _text.replace("// flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}", "")
    news_detail.append(body_text.strip())
    
    return news_detail


#기사별 공감 지표 수집
def get_like_statics(n_url):
    oid = n_url.split("oid=")[1].split("&")[0]
    aid = n_url.split("aid=")[1]
    header = {
        "User-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36",
        "referer": n_url
    }

    #Jquery URL 
    c_url = "https://news.like.naver.com/v1/search/contents?suppress_response_codes=true&callback=jQuery112405673248941186477_1550071279893&q=NEWS%5Bne_" + oid + "_" + aid + "%5D%7CNEWS_MAIN%5Bne_" + oid + "_" + aid + "%5D&isDuplication=false&_=1550071279894"
    res = requests.get(c_url, headers=header)
    cont = BeautifulSoup(res.content, "html.parser")

    like_statics = []    

    #좋아요 수치 파싱
    if not 'like","count":' in str(cont):
        like_news = 0
    else:
        like_news = str(cont).split('like","count":')[1].split(",")[0]
    like_statics.append(int(like_news))

    #슬퍼요 수치 파싱
    if not 'sad","count":' in str(cont):
        sad_news = 0
    else:
        sad_news = str(cont).split('sad","count":')[1].split(",")[0]
    like_statics.append(int(sad_news))

    #화나요 수치 파싱
    if not 'angry","count":' in str(cont):
        angry_news = 0
    else:
        angry_news = str(cont).split('angry","count":')[1].split(",")[0]
    like_statics.append(int(angry_news))

    #후속기사 원해요 수치 파싱
    if not 'want","count":' in str(cont):
        want_news = 0
    else:
        want_news = str(cont).split('want","count":')[1].split(",")[0]
    like_statics.append(int(want_news))

    #훈훈해요 수치 파싱
    if not 'warm","count":' in str(cont):
        warm_news = 0
    else:
        warm_news = str(cont).split('warm","count":')[1].split(",")[0]
    like_statics.append(int(warm_news))

    return like_statics


#댓글 수량 수집 - json 파싱 
def get_comment_num(n_url):
    oid = n_url.split("oid=")[1].split("&")[0]
    aid = n_url.split("aid=")[1]
    header = {
        "User-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36",
        "referer": n_url,
    }

    #댓글 jqeury URL
    c_url ='https://apis.naver.com/commentBox/cbox/web_neo_list_jsonp.json?ticket=news&templateId=default_society&pool=cbox5&_callback=jQuery112409540374534082234_1571293027001&lang=ko&country=KR&objectId=news{}%2C{}&categoryId=&pageSize=20&indexSize=10&groupId=&listType=OBJECT&pageType=more&page=1&initialize=true&userType=&useAltSort=true&replyPageSize=20&moveTo=&sort=favorite&includeAllStatus=true&_=1571293027002'.format(oid, aid)
    res = requests.get(c_url, headers=header)
    contents = BeautifulSoup(res.content, "html.parser")
    comments = contents.text.replace('jQuery112409540374534082234_1571293027001(','').replace(');','')
    
    comment_no = []
    
    #json 파일 변환 
    json_comments = json.loads(comments)
    
    #코멘트 수 
    comment_num = json_comments["result"]["count"]['comment']
    comment_no.append(comment_num)
    
    #코멘트 reply 수
    comment_reply_num = json_comments["result"]["count"]['reply']
    comment_no.append(comment_reply_num)

    return comment_no


#json 파싱
def get_comment(n_url):
    global total_comm
    oid = n_url.split("oid=")[1].split("&")[0]
    aid = n_url.split("aid=")[1]
    header = {
        "User-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36",
        "referer": n_url,
    }
    
    #댓글 jqeury URL
    c_url ='https://apis.naver.com/commentBox/cbox/web_neo_list_jsonp.json?ticket=news&templateId=default_society&pool=cbox5&_callback=jQuery112409540374534082234_1571293027001&lang=ko&country=KR&objectId=news{}%2C{}&categoryId=&pageSize={}&indexSize=10&groupId=&listType=OBJECT&pageType=more&page=1&initialize=true&userType=&useAltSort=true&replyPageSize={}&moveTo=&sort=favorite&includeAllStatus=true&_=1571293027002'.format(oid, aid, total_comm[0], total_comm[1])
    res = requests.get(c_url, headers=header)
    contents = BeautifulSoup(res.content, "html.parser")
    comments = contents.text.replace('jQuery112409540374534082234_1571293027001(','').replace(');','')
    
    comment_detail = []
    
    #json 파일 변환 
    json_comments = json.loads(comments)    
    comments = json_comments["result"]['commentList']
    
    for com in comments:         
        user = com["userName"]
        comment_detail.append(user)
        
        comment = com["contents"].replace('\n','')
        comment_detail.append(comment)
        
        sympathy_no = com["sympathyCount"]
        comment_detail.append(sympathy_no)
        
        antipathy_no = com["antipathyCount"]
        comment_detail.append(antipathy_no)
              
        com_date = com["regTime"][:10]     
        com_time = com["regTime"][11:19]
        date_time = com_date + ' ' + com_time
        comment_detail.append(date_time)
        
    return comment_detail
        

while page < 5:
    print("\n페이지_{}의 크롤링을 시작했습니다".format(page))
    url = "https://search.naver.com/search.naver?where=news&query={}&sort=1&ds={}&de={}&nso=so%3Ar%2Cp%3Afrom{}to{}%2Ca%3A&start={}".format(query, s_date, e_date, s_from, e_to, str(page))
    
    req = requests.get(url, headers=header)
    print("네이버 검색창 페이지_{} URL:".format(page), url)

    #html 가져오기
    html = req.content
    soup = BeautifulSoup(html, 'lxml')

    for urls in soup.select("._sp_each_url"):
        try:
            if urls["href"].startswith("https://news.naver.com"):

                #네이버 뉴스 중 엔터테인먼트 섹션에 게재된 기사 필터팅 (제외)
                if not "&sid1=106" in str(urls["href"]):
                    news_detail = get_news(urls["href"])           #네이버 뉴스 정보 가져오기
                    total_comm = get_comment_num(urls["href"])     #기사의 댓글 건수 가져오기
                    like_statics = get_like_statics(urls["href"])  #기사의 공감 지표(좋아요/슬퍼요 등) 가져오기      
                    if total_comm[0] == 0:                         #댓글 있는 기사에 한하여 댓글 정보 가져오기
                        pass
                    else:
                        naver_comments = get_comment(urls["href"])
                    df_news=df_news.append(Series([query, news_detail[0], news_detail[1], news_detail[2], news_detail[3], news_detail[4], urls["href"], total_comm[0], like_statics[0], like_statics[1],like_statics[2], like_statics[3], like_statics[4]], index=columns_news), ignore_index=True)
                    df_comments=df_comments.append(Series([urls["href"], naver_comments[0], naver_comments[1], naver_comments[2], naver_comments[3], naver_comments[4]], index=columns_comments), ignore_index=True)
            else:
                continue
                
        except Exception as e:
            print(e)
            continue
    #네이버 검색 시, 한 페이지에 기사 10건씩 검색됨
    page += 10

print('='*50)
print("기사 수집을 종료합니다")
print('='*50)


#파일 닫기



페이지_1의 크롤링을 시작했습니다
네이버 검색창 페이지_1 URL: https://search.naver.com/search.naver?where=news&query=이춘재 8차&sort=1&ds=2019.09.19&de=2019.10.18&nso=so%3Ar%2Cp%3Afrom20190919to20191018%2Ca%3A&start=1
기사 수집을 종료합니다


In [4]:
df_news

,검색어,게재일,게재시간,매체명,헤드라인,본문,URL,댓글 수,좋아요 수,슬퍼요 수,화나요 수,훈훈해요 수,후속기사 희망 수
0,이춘재 8차,2019.10.18,오후 11:11,서울신문,여야 “화성 8차사건 진실 밝혀야” 국감서 한목소리,‘고문기술자’ 이근안 당시 수사참여 의혹 제기[서울신문]18일 국회 행정안전위원회의...,https://news.naver.com/main/read.nhn?mode=LSD&...,2,0,0,0,0,0
1,이춘재 8차,2019.10.18,오후 9:18,노컷뉴스,"여야 의원들, 국감서 화성 8차 사건 진실 규명 촉구",[CBS노컷뉴스 고무성 기자](사진=연합뉴스)18일 국회 행정안전위원회의 경기남부지...,https://news.naver.com/main/read.nhn?mode=LSD&...,0,0,0,0,0,0
2,이춘재 8차,2019.10.18,오후 9:01,중앙일보,"""고문 기술자 이근안 개입했나"" 화성 8차사건 규탄 받은 경찰","""화성 연쇄살인 사건에 대해 질의하겠다."" 18일 국회 행정안전위원회의 경...",https://news.naver.com/main/read.nhn?mode=LSD&...,60,3,1,69,6,1
3,이춘재 8차,2019.10.18,오후 8:55,KBS,[국감] “‘화성 8차 사건’ 진실 밝혀라”…의원들 강력 주문,경기남부지방경찰청 국정감사에서 여야 의원들이 진범 논란에 휩싸인 화성연쇄살인 8차 ...,https://news.naver.com/main/read.nhn?mode=LSD&...,0,0,0,4,0,0
4,이춘재 8차,2019.10.18,오후 8:42,뉴스1,"[국감현장]""고문기술자 이근안, 화성 8차 사건서 고문 했을 수도""","김영호 의원 ""당시 화성경찰서 근무해""경기남부청 ""확인된 바 없지만 진위 여부 가릴...",https://news.naver.com/main/read.nhn?mode=LSD&...,1,0,0,2,0,0
5,이춘재 8차,2019.10.18,오후 8:34,연합뉴스,[국감현장] '화성 8차사건' 진실 밝혀야…여야 한목소리,과거 경찰 수사 질타·'고문기술자' 이근안 수사참여 의혹도(수원=연합뉴스) 최종호 ...,https://news.naver.com/main/read.nhn?mode=LSD&...,0,1,0,0,1,0
6,이춘재 8차,2019.10.18,오후 8:27,YTN,[뉴있저/소있저] 화성 8차 사건 '재심'...진실공방 본격화,동영상 뉴스 \t \t1986년 ~ 1991년 10차례 발생한 화성 연쇄...,https://news.naver.com/main/read.nhn?mode=LSD&...,4,1,0,0,0,1
7,이춘재 8차,2019.10.18,오후 8:22,노컷뉴스,"""화성사건 당시 고문기술자 이근안 근무했다""","김영호 의원 ""이 씨가 고문기술 전수 가능성도""경찰 ""기록상 이 씨가 수사에 참여한...",https://news.naver.com/main/read.nhn?mode=LSD&...,2,1,0,2,0,0


In [48]:
df_comments

,URL,댓글작성자,댓글,공감 수,비공감 수,댓글작성시간
0,https://news.naver.com/main/read.nhn?mode=LSD&...,azte****,"사람은 진짜관상이 그사람의모든걸말해준다 민갑룡저인간얼굴상은 우유부단과 판단,선택장애...",19,3,2019-10-18 12:26:58
1,https://news.naver.com/main/read.nhn?mode=LSD&...,janc****,진짜 무섭다.. 경찰들.. 범인 잡는게 중요하다지만 멀쩡한 사람 고문해서 허위 자백...,39,0,2019-10-18 11:31:40
2,https://news.naver.com/main/read.nhn?mode=LSD&...,sook****,민갑룡씨~ 개구리소년 실종사건 해결하겠다고 어쩌고 방송타던데 저번엔 경찰이 도박하다...,1,1,2019-10-18 12:39:20
3,https://news.naver.com/main/read.nhn?mode=LSD&...,dlsx****,견찰과 기레기들의 콜라보^^,3,0,2019-10-18 11:35:27
4,https://news.naver.com/main/read.nhn?mode=LSD&...,3ss1****,버ㄹ ㅓ지...죄없는 이를 밟고..올라간 버ㄹ ㅓ지..,426,0,2019-10-18 11:05:14
5,https://news.naver.com/main/read.nhn?mode=LSD&...,phj_****,"살인의 추억이, 경찰들의 고문의 추억. 그 당시 고문당한 무고한 시민들은 얼마나 힘...",0,0,2019-10-18 11:05:38
6,https://news.naver.com/main/read.nhn?mode=LSD&...,cis1****,만약 정말로 무죄로 판결된다면 그동안의 억울함은 무엇으로 보상받나요??,0,0,2019-10-18 07:13:45
7,https://news.naver.com/main/read.nhn?mode=LSD&...,phj_****,경찰 실수에 한표! 재심 동의 한표!,0,0,2019-10-18 07:08:19
8,https://news.naver.com/main/read.nhn?mode=LSD&...,jsy0****,이남자.보면 볼수록 매력덩어리...오늘 완전 팬 됐음.너무 멋지다.!!!조국청문회 ...,144,26,2019-10-17 22:17:44
9,https://news.naver.com/main/read.nhn?mode=LSD&...,ycre****,윤석열은 그대로인데 정치권만 자리바꿔가며 gr하지....이시대의 이순신 같음.......,2,0,2019-10-17 21:08:24


In [33]:
df_news.to_excel('naver_news.xlsx', index=False)
df_comments.to_excel('df_comments.xlsx', index=False)